# Data Science with Databricks

## ML is key to wind turbine farm optimization

The current market makes energy even more strategic than before. Being able to ingest and analyze our Wind turbine state is a first step, but this isn't enough to thrive in a very competitive market.

We need to go further to optimize our energy production, reduce maintenance cost and reduce downtime. Modern data company achieve this with AI.



# Predictive maintenance - Single click deployment with AutoML

Let's see how we can now leverage the sensor data to build a model predictive maintenance model.

Our first step as Data Scientist is to analyze and build the features we'll use to train our model.

The sensor table enriched with turbine data has been saved within our Delta Live Table pipeline. All we have to do is read this information, analyze it and start an Auto-ML run.

<img src="https://github.com/databricks-demos/dbdemos-resources/raw/main/images/manufacturing/lakehouse-iot-turbine/lakehouse-manuf-iot-ds-flow.png" width="1000px">

*Note: Make sure you switched to the "Machine Learning" persona on the top left menu.*


In [0]:
%pip install --quiet databricks-sdk==0.40.0 databricks-feature-engineering==0.8.0 mlflow==2.20.2
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false

## Data exploration and analysis

Let's review our dataset and start analyze the data we have to predict our churn

In [0]:
def plot(sensor_report):
  turbine_id = (spark.table('turbine_training_dataset')
                        .where(f"abnormal_sensor = '{sensor_report}' ")
                        .limit(1)
                        .collect()[0]['turbine_id'])
  
  df = (spark.table('sensor_bronze')
                .where(f"turbine_id == '{turbine_id}'")
                .orderBy('timestamp')
                .limit(500)
                .pandas_api())
  
  df.plot(x="timestamp", 
          y=["sensor_B"], 
          kind="line", 
          title=f'Sensor report: {sensor_report}').show()

plot('ok')

In [0]:
plot('sensor_B')

As we can see in these graph, we can clearly see some anomaly on the readings we get from sensor B. Let's continue our exploration and use the std we computed in our main feature table.

In [0]:
# Read our churn_features table:
turbine_dataset = (spark.table("turbine_training_dataset")
                        .withColumn("damaged", F.col("abnormal_sensor") != "ok"))

display(turbine_dataset)

In [0]:
# Damaged sensors clearly have a different distribution:
import seaborn as sns

g = sns.PairGrid(turbine_dataset.sample(0.01).toPandas()[['std_sensor_A', 'std_sensor_E', 'damaged', 'avg_energy']],
                 diag_sharey=False,
                 hue='damaged')

(g.map_lower(sns.kdeplot)
  .map_diag(sns.kdeplot, lw=3)
  .map_upper(sns.regplot)
  .add_legend())

### Further data analysis and preparation using pandas API

Because our Data Scientist team is familiar with Pandas, we'll use `pandas on spark` to scale `pandas` code. The Pandas instructions will be converted in the spark engine under the hood and distributed at scale.

*Note: Starting from `spark 3.2`, koalas is builtin and we can get an Pandas Dataframe using `pandas_api()`.*

In [0]:
# Convert to pandas (koalas):
dataset = turbine_dataset.pandas_api()

# Select the columns we would like to use as ML Model features. 
# Note: we removed percentiles_sensor_A/B/C.. feature to make the demo easier:
columns = ["turbine_id",
           "hourly_timestamp",
           "avg_energy",
           "std_sensor_A",
           "std_sensor_B",
           "std_sensor_C",
           "std_sensor_D",
           "std_sensor_E",
           "std_sensor_F",
           "location",
           "model",
           "state",
           "abnormal_sensor"]

dataset = dataset[columns]

# Drop missing values:
dataset = dataset.dropna()
display(dataset)


## Write to Feature Store (Optional)

<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/product_demos/mlops-end2end-flow-feature-store.png" style="float:right" width="500" />

Once our features are ready, we'll save them in Databricks Feature Store. Under the hood, features store are backed by a Delta Lake table.

This will allow discoverability and reusability of our feature accross our organization, increasing team efficiency.

Feature store will bring traceability and governance in our deployment, knowing which model is dependent of which set of features. It also simplify realtime serving.

Make sure you're using the "Machine Learning" menu to have access to your feature store using the UI.

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
fe = FeatureEngineeringClient()

try:
    spark.sql("DROP TABLE IF EXISTS turbine_hourly_features")
    fe.drop_table(name=f"{catalog}.{db}.turbine_hourly_features")
except:
    pass

churn_feature_table = fe.create_table(
    name=f"{catalog}.{db}.turbine_hourly_features",
    primary_keys=["turbine_id", "hourly_timestamp"],
    schema=dataset.spark.schema(),
    description="These features are derived from the turbine_training_dataset table in the data intelligence platform.  We made some basic transformations and removed NA value."
)

fe.write_table(df=dataset.drop_duplicates(subset=["turbine_id", "hourly_timestamp"]).to_spark(),
               name=f"{catalog}.{db}.turbine_hourly_features")

features = fe.read_table(name=f"{catalog}.{db}.turbine_hourly_features")
display(features)


## Accelerating Predictive maintenance model creation using MLFlow and Databricks Auto-ML

MLflow is an open source project allowing model tracking, packaging and deployment. Everytime your datascientist team work on a model, Databricks will track all the parameter and data used and will save it. This ensure ML traceability and reproductibility, making it easy to know which model was build using which parameters/data.

### A glass-box solution that empowers data teams without taking away control

While Databricks simplify model deployment and governance (MLOps) with MLFlow, bootstraping new ML projects can still be long and inefficient. 

Instead of creating the same boilerplate for each new project, Databricks Auto-ML can automatically generate state of the art models for Classifications, regression, and forecast.


<img width="1000" src="https://github.com/QuentinAmbard/databricks-demo/raw/main/retail/resources/images/auto-ml-full.png"/>


Models can be directly deployed, or instead leverage generated notebooks to boostrap projects with best-practices, saving you weeks of efforts.

<br style="clear: both">

<img style="float: right" width="600" src="https://github.com/QuentinAmbard/databricks-demo/raw/main/retail/resources/images/churn-auto-ml.png"/>

### Using Databricks Auto ML with our Churn dataset

Auto ML is available in the "Machine Learning" space. All we have to do is start a new Auto-ML experimentation and select the feature table we just created (`turbine_hourly_features`)

Our prediction target is the `abnormal_sensor` column.

Click on Start, and Databricks will do the rest.

While this is done using the UI, you can also leverage the [python API](https://docs.databricks.com/applications/machine-learning/automl.html#automl-python-api-1)

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
fe = FeatureEngineeringClient()

In [0]:
xp_path = "/Shared/dbdemos/experiments/lakehouse-iot-platform"
xp_name = f"automl_iot_{datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}"

training_dataset = (fe.read_table(name=f"{catalog}.{db}.turbine_hourly_features")
                        .drop("turbine_id")
                        .sample(0.1)) #Reduce the dataset size to speedup the demo

try:
    from databricks import automl
    automl_run = automl.classify(
        experiment_name = xp_name,
        experiment_dir = xp_path,
        dataset = training_dataset,
        target_col = "abnormal_sensor",
        timeout_minutes = 10
    )

    #Make sure all users can access dbdemos shared experiment
    DBDemos.set_experiment_permission(f"{xp_path}/{xp_name}")
except Exception as e:
    if "cannot import name 'automl'" in str(e):
        DBDemos.create_mockup_automl_run(f"{xp_path}/{xp_name}", training_dataset.toPandas())
    else:
        raise e

AutoML saved our best model in the MLFlow registry. Open the experiment from the AutoML run to explore its artifact and analyze the parameters used, including traceability to the notebook used for its creation.

If we're ready, we can move this model into Production stage in a click, or using the API. Let' register the model to Unity Catalog and move it to production.

You can programatically get the last best run from your automl training:
```
from mlflow import MlflowClient

# retrieve best model trial run
trial_id = automl_run.best_trial.mlflow_run_id
model_uri = "runs:/{}/model".format(automl_run.best_trial.mlflow_run_id)
#Use Databricks Unity Catalog to save our model
latest_model = mlflow.register_model(model_uri, f"{catalog}.{db}.{model_name}")
# Flag it as Production ready using UC Aliases
MlflowClient().set_registered_model_alias(name=f"{catalog}.{db}.{model_name}", alias="prod", version=latest_model.version)
```